<a href="https://colab.research.google.com/github/diem-ai/movie-recommender/blob/master/HydridModelEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Colab setup

In [1]:
#mouting google drive with colab
from google.colab import drive
# This will prompt for authorization.
# authorization code: 4/OwErfUj6QceGXhIGx_RWv0MKclb9rilw8UsJnZqFbSez-QS8zQ399JU
drive.mount('/content/drive')

#https://drive.google.com/open?id=1uGOD2rFYVL9mods0KIpU_oY2Ye6fl8KE

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# authorzing file access on google drive
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# For colab running only
my_module = drive.CreateFile({'id':'1uGOD2rFYVL9mods0KIpU_oY2Ye6fl8KE'})
my_module.GetContentFile('recommender.py')


#### 2. Import python libraries and recommender.py

In [4]:
# Import library
import pandas as pd
import recommender
from recommender import Utility as utility
from recommender import Similarity as similarity



"\nimport nltk\nnltk.download('punkt')\nnltk.download('stopwords')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('wordnet')\n"

#### 3. Loading normalised movies dataset and training data of two models

In [0]:
movies = pd.read_csv("/content/drive/My Drive/Test/normalised_movies.csv")
ratings_matrix = pd.read_csv("/content/drive/My Drive/Test/ratings_matrix.csv",  index_col=0)
content_matrix = pd.read_csv("/content/drive/My Drive/Test/content_matrix.csv",  index_col=0)

In [6]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 6 columns):
Unnamed: 0      45463 non-null int64
id              45463 non-null int64
title           45460 non-null object
genres          43021 non-null object
vote_count      45460 non-null float64
vote_average    45460 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.1+ MB
None


In [7]:
print(ratings_matrix.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7174 entries, 2 to 176143
Columns: 3721 entries, 0 to 3720
dtypes: float64(3721)
memory usage: 203.7 MB
None


In [8]:
print(content_matrix.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46629 entries, 0 to 46628
Columns: 1999 entries, 0 to 1998
dtypes: float64(1999)
memory usage: 711.5 MB
None


#### 4. Print top 10 rated movies

In [11]:
utility.get_top_n_movies(movies, 10)

,Unnamed: 0,id,title,genres,vote_count,vote_average
0,0,862,Toy Story,animation comedy family,5415.0,7.7
1,1,8844,Jumanji,adventure fantasy family,2413.0,6.9
2,2,15602,Grumpier Old Men,romance comedy,92.0,6.5
3,3,31357,Waiting to Exhale,comedy drama romance,34.0,6.1
4,4,11862,Father of the Bride Part II,comedy,173.0,5.7
5,5,949,Heat,action crime drama thriller,1886.0,7.7
6,6,11860,Sabrina,comedy romance,141.0,6.2
7,7,45325,Tom and Huck,action adventure drama family,45.0,5.4
8,8,9091,Sudden Death,action adventure thriller,174.0,5.5
9,9,710,GoldenEye,adventure action thriller,1194.0,6.6


In [13]:
movies[movies['title']=="Notting Hill"]

,Unnamed: 0,id,title,genres,vote_count,vote_average
2556,2556,509,Notting Hill,romance comedy drama,1309.0,7.0


#### 5. Make a recommendation with collaborative filtering model

In [17]:
movie_title = "Notting Hill"
features = ['title', 'genres', 'vote_average']

movie_id = utility.get_movie_by_title(movies, movie_title)
print("10 recommendations for {}".format(movie_title))
similarity.make_recommendation(movie_id, ratings_matrix, movies, features, 10)


10 recommendations for Notting Hill
                            title                               genres  \
2                Grumpier Old Men                       romance comedy   
8391               Brother Orchid   drama action comedy thriller crime   
8341                    The Ghoul  romance drama horror action mystery   
8359                 The Informer                                drama   
8363  The Crime of Monsieur Lange                          drama crime   
8366       Mr. Deeds Goes to Town                 comedy drama romance   
8367         The Petrified Forest                          crime drama   
8368        Revolt of the Zombies                               horror   
8371           Wife vs. Secretary                 comedy drama romance   

      vote_average  
2              6.5  
8391           6.7  
8341           5.0  
8359           6.5  
8363           6.2  
8366           7.3  
8367           7.1  
8368           3.0  
8371           6.5  


#### 6. Make recommendation with content based model

In [18]:

idx = utility.get_idx_by_title(movies, movie_title)
print("10 recommendations for {}".format(movie_title))
similarity.make_recommendation(idx, content_matrix, movies, features, 10)


10 recommendations for Notting Hill
                                title                    genres  vote_average
44710         Faust et Méphistophélès            fantasy horror           5.0
44695          Jupiter's Thunderballs            comedy fantasy           4.8
33572               Storm in a Teacup            comedy romance           8.1
21219              Thanks for Sharing              comedy drama           5.7
3866                   102 Dalmatians             comedy family           5.1
18389                  All Night Long               drama music           7.0
19983                  Crooked Arrows                     drama           5.6
19597        Ator, the Fighting Eagle  fantasy action adventure           3.6
42962  Timecop 2: The Berlin Decision    action science fiction           4.0
